## Topic Summary


In this project I aim to develop a generalizable novel methodology for identifying website list in a target category. And I will use "dangerous website" as an example to demonstrate my methodology. Finally I will analyze the characteristics of dangerous websites via calsual inference.

To be more specific, I will answer the following questions in this project:

* 1. ***How to ascertain whether a website belongs to a specific website category without having a dataset?***
 Not every classification comes with a "ground truth" available for our study. Therefore, I aim to develop a method to determine whether a website falls into a given category in the absence of "ground truth".(i.e. Input: Category defination, website list.  Output: Website which belongs to this category).

* 2. ***Identifying and characterizing “dangerous” websites(Or AD website):*** What specifc kinds of content and tactics in website cause the websites dangerous? In other word, what are the characteristics of dangerous websites have in common? 
 We will use "dangerous website" as an example.



This project's contribution will be:

1. Develop a generalizable novel methodology for identifying website list in a target category without ground truth.(We will use "Whether a website is dangerous" as an example. Or we can use "Whether a website is a ad website" as an example.)

2. Analyze the characteristics of dangerous websites via calsual inference.(Or other techniques, not sure yet.)





## Roadmap:

* Construct a website list which contains dangerous websites and non-dangerous websites.
    + Figure which feature is the most important for classify a website into a target category.(i.e. What feature can represent a website)
        - Collect website list (About 200,000 websites).
        - Decide which feature is better, collect website features via crawler(Now about 30 features).
    + Construct a website list without ground truth.(**Core contribution**)
        - Download some pre-trained model from Huggingface.
        - Compare the performance of different models and choose the best one as the teacher model.
        - Use the teacher-student model to train a student model.
        - Use the student model's proformance to evaluate the teacher model's performance and fine-tune the teacher model.
        - Use a well-known dataset (AD DataSet) to evaluate our model.
        - Use the model to predict the website list.
* Identifying and characterizing "dangerous" websites:
    Initially I will use causal inference to analyze the characteristics of dangerous websites. Crawling the ad, tracker and script information of the website, and then use causal inference to analyze the characteristics of dangerous websites.(***NEED TO READ MORE PAPER.***)

    ```{mermaid}
flowchart TB
    FEAT(Features) --> CRAWLAR[Crawlar]
    URL_LIST(Website URL List) --> CRAWLAR
    CRAWLAR --> FEATURE_DATA(Website Features List)
    FEATURE_DATA --> TEACHER[Teacher Model]
  subgraph border[ ]
    style border fill:#FFFFFF, stroke-dasharray: 5, 5;
    TEACHER
    TEACHER --> D[Psudo_labdel]
    D --> E[Student Model]
    E --> F(Train & Test)
    F --> ERROR(Error V1)
    I[Strong Label] --> F
  end
    TEACHER --> TEACHER_PROFORMANCE((Model Proformance))
    WEB_KB_DATASET(Web KB Dataset) --> TEACHER_PROFORMANCE
    ERROR -. "finetune" .-> TEACHER
    D -. "manually review" .-> RAEL_DANGEROUS((Dangerous website))
    style FEAT fill:#d3d3d3
    style D fill:#d3d3d3
    style I fill:#d3d3d3
    style WEB_KB_DATASET fill:#d3d3d3
    style TEACHER_PROFORMANCE fill:#a5a5a5
    style RAEL_DANGEROUS fill:#a5a5a5

```


## Process

### 1.Collect a list of websites and the data we want to use.

Now we use data from [ISCX-URL2016](https://www.unb.ca/cic/datasets/url-2016.html)

In website classification problem, we have 4 types of features to classify a website:

1. URL features: URL features are the features that can be extracted from the URL of the website. For example, the length of the URL, the presence of special characters, the presence of an IP address, etc.

2. On page features: On page features are the features that can be extracted from the content of the website. For example, the presence of certain keywords, the presence of certain HTML tags, etc.

3. JavaScript features: JavaScript features are the features that can be extracted from the JavaScript code of the website. For example, the presence of certain keywords, the presence of certain functions, etc.

4. Connection features: Connection features are other website that have a connection with the website. (***Increase computational complexity badly, do not use***)

5. User features: User features are the features that can be extracted from the user of the website. For example, the location of the user, the browser used by the user, etc. (***Do not do research on this feature. But will use in future***)


According to the reference we can use the following features to identify the category of a website:

| Feature ID | Variable Name                      | Variable Description                                                |
|------------|------------------------------------|---------------------------------------------------------------------|
| 1          | IP_Address_Usage                   | Indicates whether the IP address is used as the URL.               |
| 2          | Long_URL                           | Reflects if the URL is unusually long to hide suspicious parts.    |
| 3          | At_Symbol_In_URL                   | Denotes the presence of '@' symbol in the URL.                     |
| 4          | Domain_Prefix_Suffix               | Presence of prefix or suffix in the domain.                        |
| 5          | Sub_Domain_Count                   | Number of sub-domains and multi sub-domains.                       |
| 6          | Request_URL                        | The URL which is requested.                                        |
| 7          | Anchor_URL                         | The URL of anchor links in the webpage.                            |
| 8          | Abnormal_URL                       | Identifies if the URL structure is abnormal.                       |
| 9          | Links_In_Tags                      | Number of links present within tags (like `<a>`, `<img>` etc.).    |
| 10         | Webpage_Title                      | The title of the webpage.                                          |
| 11         | Meta_Description                   | Description of the webpage provided in meta tags (if available).  |
| 12         | Redirect_Page                      | Indicates whether the webpage redirects to another page.           |
| 13         | Active_Content_Presence            | Presence of active content like JavaScript, Flash.                 |
| 14         | Technology_Stack_Info              | Information on the technology stack used, analyzed from HTTP headers or page content. |
| 15         | HTTP_Header_Info                   | Information provided in the HTTP headers of the webpage.           |
| 16         | Privacy_Policy_Presence            | Indicates the presence of Privacy Policy/Terms of Service through text analysis. |
| 17         | Mobile_Responsiveness              | Indicates whether the webpage is designed responsively for mobile devices. |
| 18         | External_Links_Count               | Number of external links present on the webpage by analyzing `<a>` tags. |


For fetching the features above, we write a class [WebScraper](https://github.com/xinliu97/evil-detector/blob/main/evil-detector-python/WebScraper.py) to gethering those data.









### Preprocessing the data.

1. Tokenize the text data in features.